## Tensorflowにおける実装の流れ
1. モデルの定義
1. 誤差関数の定義
1. 最適化手法の定義
1. セッションの初期化
1. 学習

In [15]:
import numpy as np
import tensorflow as tf

尤度関数の偏微分を取りやすくするために対数をとって和の形にする。
<br>これを交差エントロピー誤差関数と予備、以下のようにかける
<br>この関数を最小化することがもともとの尤度関数を最適化することになる。
<br>一般的には、この関数Eのことを誤差関数(error function)、或いは損失関数(loss function)と呼ぶ


$$
E(\textbf{w},b) = - \sum_{n=1}^{N} \{t_n\log{y_n}+(1 - t_n)\log{(1 - y_n)} \}
$$

In [16]:
'''
モデル設定
'''
tf.set_random_seed(0) #乱数シード

# 入力は2次元、出力は1次元
w = tf.Variable(tf.zeros([2, 1]))
b = tf.Variable(tf.zeros([1]))

# placeholder: データを格納する入れ物。
# モデルの定義の時にはデータの次元だけ決めておき、モデルの学習など、
# 実際にデータが必要になったタイミングで値を入れて実装の式を評価することを
# 可能にしている
x = tf.placeholder(tf.float32, shape=[None, 2])
t = tf.placeholder(tf.float32, shape=[None, 1])
y = tf.nn.sigmoid(tf.matmul(x, w) + b)

# 交差エントロピー誤差関数をtensorflowの書式で記述
cross_entropy = - tf.reduce_sum(t * tf.log(y) + (1 - t) * tf.log(1 - y))
# (確率的)勾配降下法を適用
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cross_entropy)

# Y >= 0.5かどうかでニューロンが発火するかどうかを決める
correct_prediction = tf.equal(tf.to_float(tf.greater(y, 0.5)), t)

ORゲートは以下の表のようになる

|x_1|x_2|y|
|--|--|--|
|0|0|0|
|0|1|1|
|1|0|1|
|1|1|1|


In [18]:
'''モデル学習'''
# ORゲート
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
Y = np.array([[0], [1], [1], [1]])

# 初期化
# Tensorflowではセッションというデータのやり取りの流れの中で計算が行われる
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

# 学習
# sess.run(train_step)は勾配降下法による学習に相当するが、
# その際にfeed_dictを書くことによってplacefolderであるx, tに実際の値を代入している
# ここではエポック数を200に設定している
for epoch in range(200):
    sess.run(train_step, feed_dict={
        x: X,
        t: Y
    })

In [20]:
'''
学習結果の確認
'''
classfied = correct_prediction.eval(session=sess, feed_dict={
    x: X,
    t: Y
})

prob = y.eval(session=sess, feed_dict={
    x: X
})

print('classified:')
print(classfied)
print()
print('output probability')
print(prob)

classified:
[[ True]
 [ True]
 [ True]
 [ True]]

output probability
[[0.22355038]
 [0.9142595 ]
 [0.9142595 ]
 [0.99747425]]
